In [1]:
# 0) bootstrap
import os, sys
from pathlib import Path

ROOT = Path.cwd()
while not (ROOT / "pyproject.toml").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
os.chdir(ROOT)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
print("Project root:", ROOT)

Project root: d:\IIT BBS\Job Resources\Business Optima\new-pdf-agent


In [2]:
# 1) load config & doc id
from packages.core_config.config import load_yaml

cfg = load_yaml("configs/providers.yaml", "configs/pipelines/generic_legal.yaml")
doc_id = "NFS_2019"

ft = cfg.get("finetune", {})
base_id   = str(ft.get("base_model_id", "TinyLlama/TinyLlama-1.1B-Chat-v1.0"))
base_dir  = Path(ft.get("base_model_local_dir", "models/llm/TinyLlama-1.1B-Chat-v1.0"))
dl_conf   = ft.get("download", {})
datasets_root = Path(ft.get("datasets_root", "data/datasets"))
output_root   = Path(ft.get("output_root", "data/models"))

print("Base model ID:", base_id)
print("Base model dir:", base_dir)

Base model ID: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Base model dir: models\llm\TinyLlama-1.1B-Chat-v1.0


In [3]:
# 2) optional: ensure download (runs only if folder missing or empty)
# from scripts.download_hf_model import ensure_downloaded, HFDownloadSpec

# if dl_conf.get("allow", True):
#     ensure_downloaded(HFDownloadSpec(
#         repo_id=base_id,
#         local_dir=base_dir,
#         revision=dl_conf.get("revision"),
#         ignore_patterns=dl_conf.get("ignore_patterns"),
#     ))
#     print("✓ Model ready at:", base_dir.resolve())
# else:
#     print("Download disabled; expecting model already present at:", base_dir)

In [ ]:
# 3) submit stub job (no heavy training)
from packages.finetune.submit_job import submit_job, FinetuneConfig
from packages.finetune.track_status import read_status
from packages.finetune.register_model import register_model

cfg_ft = FinetuneConfig(
    backend=str(ft.get("backend", "local_peft")),
    enable_training=bool(ft.get("enable_training", True)),
    output_root=str(output_root),
    datasets_root=str(datasets_root),
    doc_id=doc_id,
    # model resolution: let code prefer local_dir if exists
    base_model_id=base_id,
    base_model_local_dir=str(base_dir),
    # LoRA params (passed through; used only if enable_training: true)
    lora_r=int(ft.get("lora", {}).get("r", 8)),
    lora_alpha=int(ft.get("lora", {}).get("alpha", 16)),
    lora_dropout=float(ft.get("lora", {}).get("dropout", 0.05)),
    target_modules=tuple(ft.get("lora", {}).get("target_modules", ["q_proj","v_proj"])),
    # Train params (used only if enable_training: true)
    learning_rate=float(ft.get("train", {}).get("learning_rate", 2e-4)),
    weight_decay=float(ft.get("train", {}).get("weight_decay", 0.0)),
    max_steps=int(ft.get("train", {}).get("max_steps", 20)),
    per_device_train_batch_size=int(ft.get("train", {}).get("per_device_train_batch_size", 1)),
    gradient_accumulation_steps=int(ft.get("train", {}).get("gradient_accumulation_steps", 4)),
    bf16=bool(ft.get("train", {}).get("bf16", False)),
    fp16=bool(ft.get("train", {}).get("fp16", False)),
    seed=int(ft.get("train", {}).get("seed", 42)),
)

job = submit_job(cfg_ft)
print(job)

state = read_status(job["run_dir"])
state

Tokenizing train:   0%|          | 0/71 [00:00<?, ? examples/s]

Tokenizing eval:   0%|          | 0/7 [00:00<?, ? examples/s]

The 8-bit optimizer is not available on your device, only available on CUDA for now.
d:\IIT BBS\Job Resources\Business Optima\new-pdf-agent\packages\finetune\submit_job.py:271: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
d:\Anaconda\envs\pdf-agent-2\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
2,2.844100
4,3.051900
6,3.286400
8,2.274400
10,2.773400
12,2.742300
14,2.530100
16,2.417000
18,2.329800
20,2.685400


{'id': 'c0e8976c', 'doc_id': 'NFS_2019', 'created_at': 1758184431, 'status': 'COMPLETED', 'backend': 'local_peft', 'enable_training': True, 'base_model': 'models\\llm\\TinyLlama-1.1B-Chat-v1.0', 'run_dir': 'data\\models\\NFS_2019\\20bb948f', 'artifacts': {'adapter_path': 'data\\models\\NFS_2019\\20bb948f\\adapter'}, 'config': {'backend': 'local_peft', 'enable_training': True, 'output_root': 'data\\models', 'datasets_root': 'data\\datasets', 'doc_id': 'NFS_2019', 'base_model_id': 'TinyLlama/TinyLlama-1.1B-Chat-v1.0', 'base_model_local_dir': 'models\\llm\\TinyLlama-1.1B-Chat-v1.0', 'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.05, 'target_modules': ['q_proj', 'v_proj'], 'learning_rate': 0.0002, 'weight_decay': 0.0, 'max_steps': 20, 'per_device_train_batch_size': 1, 'gradient_accumulation_steps': 4, 'bf16': False, 'fp16': False, 'seed': 42}}


{'id': 'c0e8976c',
 'doc_id': 'NFS_2019',
 'created_at': 1758184431,
 'status': 'COMPLETED',
 'backend': 'local_peft',
 'enable_training': True,
 'base_model': 'models\\llm\\TinyLlama-1.1B-Chat-v1.0',
 'run_dir': 'data\\models\\NFS_2019\\20bb948f',
 'artifacts': {'adapter_path': 'data\\models\\NFS_2019\\20bb948f\\adapter'},
 'config': {'backend': 'local_peft',
  'enable_training': True,
  'output_root': 'data\\models',
  'datasets_root': 'data\\datasets',
  'doc_id': 'NFS_2019',
  'base_model_id': 'TinyLlama/TinyLlama-1.1B-Chat-v1.0',
  'base_model_local_dir': 'models\\llm\\TinyLlama-1.1B-Chat-v1.0',
  'lora_r': 8,
  'lora_alpha': 16,
  'lora_dropout': 0.05,
  'target_modules': ['q_proj', 'v_proj'],
  'learning_rate': 0.0002,
  'weight_decay': 0.0,
  'max_steps': 20,
  'per_device_train_batch_size': 1,
  'gradient_accumulation_steps': 4,
  'bf16': False,
  'fp16': False,
  'seed': 42}}

In [6]:
# 4) register (so the router can pick it up later)
if ft.get("register", True) and state["status"] == "COMPLETED":
    rec = register_model(
        doc_id=doc_id,
        base_model=state["base_model"],
        adapter_path=state["artifacts"].get("adapter_path"),
        run_dir=job["run_dir"],
        profile="generic",
    )
    print("Registered:", rec)
else:
    print("Skipped register; status:", state["status"])

Registered: {'id': 'b0b53f84', 'doc_id': 'NFS_2019', 'profile': 'generic', 'base_model': 'models\\llm\\TinyLlama-1.1B-Chat-v1.0', 'adapter_path': 'data\\models\\NFS_2019\\20bb948f\\adapter', 'run_dir': 'data\\models\\NFS_2019\\20bb948f', 'created_at': 1758186420, 'extra': {}}
